In [1]:
#https://shotanuki.com/python%E3%81%A7pdf%E3%82%92%E7%94%BB%E5%83%8F%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B/
#PythonでPDFを画像に変換する
from pdf2image import convert_from_path
import datetime
import os
now = datetime.date.today()

# カレントディレクトリの全画像をループする
for pdfname in os.listdir('./mergedTRF/'):# ❶
    ext = os.path.splitext(pdfname)
    #if ext == '.pdf':
    images = convert_from_path('./mergedTRF/'+pdfname)
    os.makedirs('./trf/'+str(now), exist_ok=True)
    i = 0
    for image in images:
        image.save('./trf/'+str(now) +'/trf_' + str(pdfname)+'_'+str(i)+'.png', 'png')
        i += 1
       

In [2]:
checkmark = [
    (1185,1235,95,145),#US 1
    (1185,1235,260,310),#LMP 2
    (1185,1235,455,505), #IVF 
    (1185,1235,645,695),#PE 
    (1280,1330,65,115), #自然妊娠 
    (1330,1380,65,115), #新鮮胚利用
    (1380,1430,60,110), #融解胚利用
 (1425,1475,60,110), #提供卵子利用
    # 4 胎児数
(1725,1775,80,130), #単胎 10
(1730,1780,215,265), #双胎
    # 5 人種
 (1860,1910,55,105), # Japanese
 (1905,1955,55,105), # Caucasian
(1945,1995,55,105),# African American
(1990,2040,55,105), # Other
    # IDDM
 (995,1045,940,990), # OIDDM No
 (995,1045,1120,1170), # OIDDM Yes
    #T21
 (1145,1195,940,990), # DS No
(1145,1195,1120,1170), # DS Yes
    #NTD
 (1585,1635,935,985), # DS No 20
 (1665,1715,935,985), # DS yes
    #NTD relation
 (1785,1835,1070,1120), # Patient herself
 (1835,1885,1070,1120), # Husbond of Patient 
(1880,1930,1070,1120), # Mother of
 (1925,1975,1070,1120), # siblings 25
 (1970,2020,1070,1120), # previous child 26
    ]

In [3]:
old_checkmark = [
    (1155,1205,175,225),#US 1
    (1155,1205,325,375),#LMP 2
    (1155,1205,510,560), #IVF 
    (1155,1205,700,750),#PE 
    # 4 胎児数
(1725,1775,80,130), #単胎 10
(1730,1780,215,265), #双胎
    # 5 人種
 (1895,1945,55,105), # Japanese
 (1895,1945,300,350), # Caucasian
(1895,1945,565,615),# African American
(1980,2030,55,105), # Other
    # IDDM
 (995,1045,940,990), # OIDDM No
 (995,1045,1120,1170), # OIDDM Yes
    #T21
 (1135,1185,940,990), # DS No
(1135,1185,1120,1170), # DS Yes
    #NTD
 (1315,1365,935,985), # NTD No 20
 (1315,1365,1135,1185), # NTD yes
    #NTD relation
     (1485,1535,1125,1175), # NTD
 (1700,1750,1125,1175), # Patient herself
 (1700,1750,1325,1375), # Husbond of Patient 
    (1835,1885,1125,1175), # Mother of Patient 
(1840,1890,1125,1175), # Mother of hus
    (1920,1970,1125,1175), #sib of hus
 (1920,1970,1325,1375), # siblings 25
 (1975,2025,1125,1175), # previous child 26
    ]

In [4]:
#PythonでOCRがんばる(1) hhttp://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916040943/
#tesseractでOCR〜数字だけでも〜 http://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916052946/
# p87 17.3.2 ステップ2：全ファイルをループして画像を開く https://www.oreilly.co.jp/pub/9784873117782/auto-sample.pdf

from PIL import Image
import cv2
import sys,os
import pyocr
import pyocr.builders
import csv
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

%matplotlib inline
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

os.makedirs('./results/checkmark_results/'+ str(now), exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/Error', exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/N', exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/Y', exist_ok = True)
os.makedirs('./csv/'+ str(now), exist_ok = True)
os.makedirs('./results/PtID_results/'+ str(now), exist_ok = True)

# カレントディレクトリの全画像をループする
for filename in os.listdir('./trf/'+ str(now)+'/'):# ❶
    ext = os.path.splitext(filename)
    if ext == '.png' or u'.jpeg' or u'.jpg':
        img = Image.open('./trf/'+ str(now)+'/' + filename)  # ❹
        img = img.crop((800,129,1050,200))
        img.save('./results/PtID_results/' + str(now)+'/' + "PtID_" + str(filename))

        tools = pyocr.get_available_tools()
        if len(tools) == 0:
            print("No OCR tool found")
            sys.exit(1)
        # The tools are returned in the recommended order of usage
        # pyocr http://motojapan.hateblo.jp/entry/2018/03/12/094636
        tool = tools[0] 
        txt = tool.image_to_string(
            Image.open('./results/PtID_results/'+ str(now)+'/' +  "PtID_" + str(filename)),
            lang="eng",
            builder=pyocr.tesseract.DigitBuilder(tesseract_layout=6),
        )
            # 6 = Assume a single uniform block of text.            
        #img.save('./results/PtID_results/' + str(now)+'/' + "PtID_" + str(now_time)+'_'+ str(filename))
        path_w = './csv/'+ str(now)+'/'+str(now)+'_'+str(now_time)+'.csv'
        with open(path_w, mode='a') as f:
            f.write(txt+'\t') 
            f.write(filename+'\t')
        img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
        # モデルを読込 --- (*1)
        labels = ["Error ", "N ", "Y "]
        model = load_model('CNN-mk-model.h5')
        # 重みデータを読込 --- (*2)
        model.load_weights('CNN-mk-weight.h5')
        img = img[1785:1835,1070:1120]
        res = model.predict(np.array([img]), batch_size=32)
        idx = res.argmax()
            
        if (idx == 0):
            for j, pt in enumerate(old_checkmark):
                x,y,w,h = pt
                img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
                img = img[x:y,w:h]
                res = model.predict(np.array([img]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./results/checkmark_results/' + str(now) + '/'+ str(labels[idx].strip())+\
                                    '/' + str(now)+ 'mk_'+ str(j)+'_'+ filename, img)
                j += 1
                with open(path_w, mode='a') as f:
                    f.write(str(labels[idx])+'\t')
        else:
            for k, pt in enumerate(checkmark):
                x,y,w,h = pt
                img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
                img = img[x:y,w:h]
                res = model.predict(np.array([img]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./results/checkmark_results/' + str(now) + '/'+ str(labels[idx].strip())+\
                                '/' + str(now)+ 'mk_'+ str(k)+'_'+ filename, img)
                k += 1
                with open(path_w, mode='a') as f:
                    f.write(str(labels[idx])+'\t')
        with open(path_w, mode='a') as f:
            f.write('\n')
        with open(path_w) as f:
            print(f.read()) 
import shutil
shutil.move('./results/PtID_results/' + str(now)+'/','./results/PtID_results/'+str(now)+'_'+ str(now_time)+'/')
shutil.move('./results/checkmark_results/' + str(now) + '/','./results/checkmark_results/' + str(now) + '_'+str(now_time)+'/')
shutil.move('./trf/'+str(now)+'/','./finished_trf/'+str(now)+'_'+str(now_time)+'/')

Using TensorFlow backend.


2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	

2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	

2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	

2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y

2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


...

2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


2003291	trf_41.pdf_5.png	N 	Y 	Y 	Error 	N 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	Y 	N 	Y 	Y 	Y 	N 	N 	N 	N 	N 	N 	
2732755 -	trf_21.pdf_0.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2744099 -	trf_11.pdf_1.png	Y 	N 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2662855	trf_41.pdf_1.png	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	N 	N 	N 	
21722420-	trf_21.pdf_7.png	N 	Y 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_9.png	Y 	Y 	N 	N 	Y 	Y 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_0.png	N 	Y 	N 	N 	Y 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
	trf_71.pdf_1.png	Y 	N 	N 	N 	Y 	N 	N 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2729660 -	trf_91.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2739662 -	trf_91.pdf_3.png	Y 	Y 	N 	N 	Y 	Y 	Y 	Y 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	


'./finished_trf/2019-08-21_8-56-50/'